In [1]:
import os
os.chdir('E:\学习\python\py_codbase\POPs_LLM')

In [2]:
# nodes
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List,Dict
from PK_LLM_endfront.loggs import logger

In [16]:
def get_first_chain()->Runnable:
    llm = ChatOpenAI(temperature=0.9)
    template = """
# Role：环境领域问答机器人

## Background：该角色是一个专门为环境领域专业人士设计的问答机器人，能够提供关于气候变化、可持续发展、环境保护法规等方面的最新研究成果和政策解读。

## Attention：请确保您的回答准确、深入，并且包含最新的研究成果。

## Constraints:
- 确保回答的准确性和深度。
- 包含最新的研究成果和数据分析。
- 回答风格适合专业人士。

## Goals:
- 提供关于气候变化、可持续发展、环境保护法规，环境污染物等方面的专业信息。
- 包含最新的研究成果。
- 解读相关政策，并提供专业见解。

## Skills:
- 精通环境科学和相关政策。
- 熟悉最新的环境研究动态。

## Workflow:
1. 接收专业人士关于环境领域的问题。
2. 分析问题，确定回答的思路
3. 提供最新的研究成果、数据和政策信息。
4. 整合信息，提供准确、深入的回答。

## OutputFormat:
- 使用和问题相同的语言回答
- 回答应包含问题的核心内容。
- 提供最新的研究成果和数据分析。
- 包含对相关政策的解读和见解。

## Suggestions:
- 确保回答的准确性和深度。
- 提供最新的研究成果和数据分析。
- 包含对相关政策的解读和见解。

## Initialization
作为一个环境领域的问答机器人，你必须专注于提供关于气候变化、可持续发展、环境保护法规等主题的准确深入的信息，遵守约束条件，遵循工作流程，并运用你的技能生成符合输出格式的内容。

# 问题
{question}
"""
    prompt = PromptTemplate(
        input_variables=['question'],
        template=template
    )
    class output_schema(BaseModel):
        answer:str = Field(description='答案')
    return prompt |llm.with_structured_output(output_schema)

def get_intent_Chain()->Runnable:
    llm = ChatOpenAI(temperature=0.2)
    template = """
您是一个问题重写器，可以结合对话历史将输入问题改写为一个更好的版本， 
***
用户问题：{question}
对话历史: {history}
"""
    class intent_output(BaseModel):
        question:str = Field(description="优化版本的问题")
    
    prompt = PromptTemplate(
        input_variables=['question','history'],
        template=template
    )
    return prompt|llm.with_structured_output(intent_output)
    

def First_Answer(input:Dict)->dict:
    """初始回答器
    Args:
        input (dict): {'question','history'}
    Returns:
        dic
    """
    intent_chain = get_intent_Chain()
    fc = get_first_chain()
    q_schema = intent_chain.invoke({'question':input['question'],'history':input['history']})
    q = q_schema.question
    print(q)
    aa = fc.invoke({'question':q})
    return {'question':q,'fa':aa.answer}

In [19]:
dic =First_Answer({'question':'如何处理这种化合物的污染','history':'什么是PFAS'})

如何处理PFAS化合物的污染


In [20]:
dic

{'question': '如何处理PFAS化合物的污染',
 'fa': '处理PFAS化合物的污染需要采取多种综合措施。首先，可以采用物理方法如吸附、膜分离等将PFAS化合物从水体中去除。其次，化学方法如氧化、还原、光降解等也可用于降解PFAS化合物。另外，生物方法如生物吸附、生物降解等也是一种有效的处理方式。综合利用各种方法，可以有效减少PFAS化合物的污染，并保护环境和人类健康。'}

In [29]:
from langchain.output_parsers import StructuredOutputParser
def Second_answer(input:Dict)->Dict:
    prompt = PromptTemplate(
        input_variables=['question','fa'],
        template="""
请你完善此答案，使其更加详细，并且有逻辑
问题：{question}
答案：{fa}
"""
    )
    llm = ChatOpenAI(model='gpt-4',temperature=0.9)
    chain = prompt|llm 
    a = chain.invoke({'question':input['question'],'fa':input['fa']}).content
    return {'a':a}

In [30]:
Second_answer(dic)